# Cubic Interpolation
Using the forward kinematics data from before, I will be using cubic interpolation to produce a smooth curve of the vector coordinates that I had previously recorded. 
***Before doing this, I must have made an n-body simulation of planets and done some other piece of code to get started with modelling (in a short timeframe).

### Intention
To produce a function that maintains the certain essence of a "shape". To be applied to the "shape" of the spine later on, but for this it could something *simpler* like one period of a sine wave (or something that looks similar to that). Trying to adjust something into that would be the challenge of the coded side of this since the checking if this final function -- to be displayed using {some numpy function} -- is within a specified range seems pretty easy (although potentially computationally intensive if I code inefficiently).

## Sources & Orders of Business
https://www.youtube.com/watch?v=9R5UxtJQNNg
1. Understand what cubic interpolation is and do some maths around this. Try and learn it well and explain it (to myself obviously).

2. Work on static cubic interpolation between varied points and try to analyse the shapes that they produce (from how I have parametrised the curves). 
    - Maybe use databases (or something from the CompSci course) to try and store the functions that look like the desired shape that I want (sine curve, one period) and see what the patterns are for that

https://uk.pinterest.com/pin/1337074888365165/
- Unironically very similar to what I want to do so... have this as some inspiration for how I could visualise my final project